In [1]:
import os
import numpy as np
import pandas as pd
from ChainLadder import ChainLadder
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
일반_출재_계약 = pd.read_excel('./data/충격시나리오/일반_출재_계약_20190812.xlsx', dtype={'UY': str, 'FY': str, 'AY': str, 'RRNR_DAT_DVCD': str})
일반_출재_보상 = pd.read_excel('./data/충격시나리오/일반_출재_보상_20190812.xlsx', dtype={'UY': str, 'FY': str, 'AY': str, 'RRNR_DAT_DVCD': str, 'RN_PRM': float, 'RN_ELP_PRM': float })
보험가격_위험계수 = pd.read_excel('./data/충격시나리오/충격시나리오_기초정보.xlsx', sheet_name='보험가격위험계수')
준비금_위험계수 = pd.read_excel('./data/충격시나리오/충격시나리오_기초정보.xlsx', sheet_name='준비금위험계수')
기준손해율 = pd.read_excel('./data/충격시나리오/충격시나리오_기초정보.xlsx', sheet_name='기준손해율')
특약별_정산수수료 = pd.read_excel('./data/충격시나리오/특약별_정산수수료.xlsx', dtype={'UY': str})
일반_특약_정보 = pd.read_excel('./data/충격시나리오/일반_특약정보_20190811.xlsx')

In [4]:
일반_출재_보상_가공 = 일반_출재_보상.query('AY != " "')
일반_출재_보상_가공 = 일반_출재_보상_가공.loc[~일반_출재_보상_가공['AY'].isna()]
일반_출재_보상_가공 = 일반_출재_보상_가공.loc[~일반_출재_보상_가공['UY'].isna()]
일반_출재_보상_가공['DY'] = (일반_출재_보상_가공['FY'].astype(int) - 일반_출재_보상_가공['AY'].astype(int) + 1)
일반_출재_보상_가공 = 일반_출재_보상_가공.query('DY >= 1')
일반_출재_보상_가공 = 일반_출재_보상_가공.query('RRNR_DAT_DVCD != "04"')
일반_출재_보상_가공 = 일반_출재_보상_가공.groupby(['NTNL_CTRY_CD', 'PDGR_CD', 'DY', 'AY'])[['RN_OGL_BNF', 'RN_OST_AMT']].sum().reset_index()
일반_출재_보상_가공['RN_LOSS'] = 일반_출재_보상_가공['RN_OGL_BNF'] + 일반_출재_보상_가공['RN_OST_AMT']

In [5]:
# 추가진전율
기준년도 = '2018'

result = []
for 상품군코드 in 준비금_위험계수['보종코드']:

    n = 7 if 상품군코드 in (25, 26) else 5
    시작년도 = str(int(기준년도) - n + 1)

    # 삼각형 생성
    일반_출재_보상_보종 = 일반_출재_보상_가공.query('NTNL_CTRY_CD == "KR"').query('PDGR_CD == @상품군코드')
    일반_출재_보상_보종 = 일반_출재_보상_보종.query('AY >= @시작년도').query('DY <= @n')
    
    삼각형_손해액_보종 = 일반_출재_보상_보종.pivot_table(index='AY', columns='DY', values='RN_LOSS', aggfunc=np.sum)
    삼각형_손해액_보종.index.name = 'Accident'
    삼각형_손해액_보종.columns.name = 'Development'

    삼각형_보험금_보종 = 일반_출재_보상_보종.pivot_table(index='AY', columns='DY', values='RN_OGL_BNF', aggfunc=np.sum)
    삼각형_보험금_보종.index.name = 'Accident'
    삼각형_보험금_보종.columns.name = 'Development'

    일반_출재_계약_가공 = 일반_출재_계약.query('RRNR_DAT_DVCD != "04"').groupby(['NTNL_CTRY_CD', 'PDGR_CD', 'FY'])[['RN_ELP_PRM']].sum().reset_index()
    일반_출재_계약_보종 = 일반_출재_계약_가공.query('NTNL_CTRY_CD == "KR"').query('PDGR_CD == @상품군코드')
    일반_출재_계약_보종 = 일반_출재_계약_보종.query('FY >= @시작년도')

    cl_loss = ChainLadder(삼각형_손해액_보종, type='incr')
    cl_paid = ChainLadder(삼각형_보험금_보종, type='incr')
    
    위험계수 = 준비금_위험계수.loc[준비금_위험계수['보종코드'] == 상품군코드, '준비금위험계수'].values[0]
    진전계수 = cl_loss.dev_factor('weighted average', 1)
    최종손해액_손해액진전_보종 = cl_loss.develop(진전계수)['Ult'].to_frame('Value')
    최종손해액_보험금진전_보종 = cl_paid.develop(진전계수)['Ult'].to_frame('Value')
    기지급보험금_보종 = cl_paid.triangle_incr.fillna(0).sum(axis=1).to_frame('Value')
    지급준비금_보종 = 최종손해액_손해액진전_보종 - 기지급보험금_보종
    최종손해율_보종 = 최종손해액_손해액진전_보종/일반_출재_계약_보종['RN_ELP_PRM'].values.reshape(-1, 1)
    
    최종손해액_준비금충격1_보종 = 지급준비금_보종*위험계수 + 최종손해액_손해액진전_보종
    최종손해율_준비금충격1_보종 = 최종손해액_준비금충격1_보종/일반_출재_계약_보종['RN_ELP_PRM'].values.reshape(-1, 1)
    추가진전율1_보종 = 최종손해율_준비금충격1_보종-최종손해율_보종
    
    최종손해액_준비금충격2_보종 = (최종손해액_손해액진전_보종-최종손해액_보험금진전_보종)*위험계수 + 최종손해액_손해액진전_보종
    최종손해율_준비금충격2_보종 = 최종손해액_준비금충격2_보종/일반_출재_계약_보종['RN_ELP_PRM'].values.reshape(-1, 1)
    추가진전율2_보종 = 최종손해율_준비금충격2_보종-최종손해율_보종
    
    추가진전율_보종 = 추가진전율1_보종.merge(추가진전율2_보종, on='Accident', how='outer')
    추가진전율_보종.insert(0, 'PDGR_CD', 상품군코드)

    result.append(추가진전율_보종)
df = pd.concat(result).reset_index()
df.columns = ['사고년도', '보종코드', '추가진전율1', '추가진전율2']

In [6]:
# 특약 보종 UY 출재경과보험료 및 출재손해율
특약명_전체 = ['재물보험특약', '기술보험_일반', '기술보험_ALOP', '근재보험특약', '배상책임보험특약', '해외PST']
result = []
for 특약명 in 특약명_전체:

    산출기간 = 7 if 특약명 == '근재보험특약' or 특약명 == '배상책임보험특약' else 5
    UY_전체 = np.arange(int(기준년도)-산출기간+1, int(기준년도)+1).astype(str)
    
    for UY in UY_전체:

        특약코드 = 일반_특약_정보.loc[일반_특약_정보['TTY_CD_NM'].str.contains(특약명), 'RRNR_TTY_CD']
        특약코드_UY = 특약코드.loc[특약코드.str.contains('U{}'.format(UY))]

        출재계약_특약_UY = 일반_출재_계약.query('RRNR_TTY_CD in @특약코드_UY').query('UY == @UY')
        출재보상_특약_UY = 일반_출재_보상.query('RRNR_TTY_CD in @특약코드_UY').query('UY == @UY')
        보종코드목록 = 출재계약_특약_UY.PDGR_CD.drop_duplicates()

        for 보종코드 in 보종코드목록:
            출재계약_특약_UY_보종 = 출재계약_특약_UY.query('PDGR_CD == @보종코드')
            출재보상_특약_UY_보종 = 출재보상_특약_UY.query('PDGR_CD == @보종코드')
            
            # 출재 계약 (보험료)
            출재미경과_UY_국내_원수_보종 = 일반_출재_계약.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_PRM'].sum()
            출재보험료_특약_UY_보종 = 출재계약_특약_UY_보종['RN_PRM'].sum()

            # 출재 계약 (경과보험료)
            출재경과보험료_UY_국내_원수_보종 = 일반_출재_계약.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_ELP_PRM'].sum()
            출재경과보험료_특약_UY_보종 = 출재계약_특약_UY_보종['RN_ELP_PRM'].sum()

            # 출재 보상 (보험금)
            출재보험금_UY_국내_원수_보종 = 일반_출재_보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_OGL_BNF'].sum()
            출재보험금_특약_UY_보종 = 출재보상_특약_UY_보종['RN_OGL_BNF'].sum()

            # 출재 보상 (준비금)
            출재준비금_UY_국내_원수_보종 = 일반_출재_보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_OST_AMT'].sum()
            출재준비금_특약_UY_보종 = 출재보상_특약_UY_보종['RN_OST_AMT'].sum()

            # 출재 보상 (손해조사비)
            출재손조비_UY_국내_원수_보종 = 일반_출재_보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['WNCR_CNV_RN_DAG_IVMT'].sum()
            출재손조비_특약_UY_보종 = 출재보상_특약_UY_보종['WNCR_CNV_RN_DAG_IVMT'].sum()

            # 출재 보상 (손해액)
            출재손해액_특약_UY_보종 = 출재보험금_특약_UY_보종 + 출재준비금_특약_UY_보종 + 출재손조비_특약_UY_보종

            # 출재 보상 (손해율)
            출재손해율_특약_UY_보종 = 출재손해액_특약_UY_보종/출재경과보험료_특약_UY_보종
            출재손해율_특약_UY_보종

            컬럼 = ['특약명', 'UY', '보종코드', '출재보험료', '출재경과보험료', '출재미경과보험료', '출재손해액', '출재손해율']
            데이터 = [특약명, UY, 보종코드, 출재보험료_특약_UY_보종, 출재경과보험료_특약_UY_보종, 출재보험료_특약_UY_보종-출재경과보험료_특약_UY_보종, 출재손해액_특약_UY_보종, 출재손해율_특약_UY_보종]
            dg = pd.DataFrame(데이터, 컬럼).T
            result.append(dg)
            
dg = pd.concat(result)

In [7]:
# 수수료 증감
df2 = dg.merge(df, left_on=['UY', '보종코드'], right_on=['사고년도', '보종코드'], how='left')
df2 = df2.merge(보험가격_위험계수, on='보종코드', how='left')
df2 = df2.merge(기준손해율[['보종코드', '기준손해율']],on='보종코드', how='left')
df2 = df2.dropna(axis=0) # 근재보험특약/종합보종은 5년까지 밖에 없음
df2 = df2.merge(특약별_정산수수료, on=['특약명', 'UY'], how='left').drop(['정산식', '사고년도'], axis=1)

# 보험가격 충격
df2['기준손해율_보험가격충격'] = df2['기준손해율']*(1+df2['보험가격위험계수'])
df2['수수료_보험가격실적기준'] = df2['출재미경과보험료']*np.fmax(np.fmin(df2['Max'], df2['기울기']*(df2['a']-df2['기준손해율'])+df2['b']), df2['Min'])
df2['수수료_보험가격충격기준'] = df2['출재미경과보험료']*np.fmax(np.fmin(df2['Max'], df2['기울기']*(df2['a']-df2['기준손해율_보험가격충격'])+df2['b']), df2['Min'])
df2['수수료증감_보험가격충격'] = df2['수수료_보험가격충격기준'] - df2['수수료_보험가격실적기준']

# 준비금 충격
df2['출재손해율_준비금충격1'] = df2['출재손해율'] + df2['추가진전율1']
df2['출재손해율_준비금충격2'] = df2['출재손해율'] + df2['추가진전율2']
df2['수수료_준비금실적기준'] = df2['출재경과보험료']*np.fmax(np.fmin(df2['Max'], df2['기울기']*(df2['a']-df2['출재손해율'])+df2['b']), df2['Min'])
df2['수수료_준비금충격1기준'] = df2['출재경과보험료']*np.fmax(np.fmin(df2['Max'], df2['기울기']*(df2['a']-df2['출재손해율_준비금충격1'])+df2['b']), df2['Min'])
df2['수수료_준비금충격2기준'] = df2['출재경과보험료']*np.fmax(np.fmin(df2['Max'], df2['기울기']*(df2['a']-df2['출재손해율_준비금충격2'])+df2['b']), df2['Min'])
df2['수수료증감_준비금충격1'] = df2['수수료_준비금충격1기준'] - df2['수수료_준비금실적기준']
df2['수수료증감_준비금충격2'] = df2['수수료_준비금충격2기준'] - df2['수수료_준비금실적기준']
df2

,특약명,UY,보종코드,출재보험료,출재경과보험료,출재미경과보험료,출재손해액,출재손해율,추가진전율1,추가진전율2,...,수수료_보험가격실적기준,수수료_보험가격충격기준,수수료증감_보험가격충격,출재손해율_준비금충격1,출재손해율_준비금충격2,수수료_준비금실적기준,수수료_준비금충격1기준,수수료_준비금충격2기준,수수료증감_준비금충격1,수수료증감_준비금충격2
0,재물보험특약,2014,23,14398997249,14398997249,0,5477050241,0.38,0.00,0.00,...,0.00,0.00,0.00,0.38,0.38,"6,964,769,639.41","6,964,769,639.41","6,964,769,639.41",0.00,0.00
1,재물보험특약,2014,27,10826011213,10826011213,0,3279273379,0.30,0.00,0.00,...,0.00,0.00,0.00,0.30,0.30,"5,907,478,132.64","5,902,327,940.92","5,902,327,940.92","-5,150,191.73","-5,150,191.73"
2,재물보험특약,2015,23,15224918955,15224918955,0,6006199733,0.39,0.00,0.00,...,0.00,0.00,0.00,0.39,0.39,"7,222,726,188.05","7,217,755,462.86","7,217,755,462.86","-4,970,725.19","-4,970,725.19"
3,재물보험특약,2015,27,11349775155,11349775155,0,5666648213,0.50,0.00,0.00,...,0.00,0.00,0.00,0.50,0.50,"4,433,003,802.05","4,423,000,862.33","4,419,982,088.37","-10,002,939.72","-13,021,713.68"
4,재물보험특약,2016,23,11925986697,11881681143,44305554,4002959476,0.34,0.05,0.05,...,"15,949,999.44","15,949,999.44",0.00,0.38,0.39,"6,207,923,884.46","5,773,161,811.50","5,741,115,976.30","-434,762,072.95","-466,807,908.16"
5,재물보험특약,2016,27,12799144333,12768542869,30601464,9898088160,0.78,0.01,0.02,...,"11,016,527.04","11,016,527.04",0.00,0.78,0.80,"4,596,675,432.84","4,596,675,432.84","4,596,675,432.84",0.00,0.00
6,재물보험특약,2017,23,11301355667,11073114567,228241100,4297969582,0.39,0.02,0.02,...,"78,058,456.20","77,601,974.00","-456,482.20",0.40,0.41,"5,663,724,508.47","5,522,334,010.15","5,469,381,346.95","-141,390,498.33","-194,343,161.52"
7,재물보험특약,2017,27,13217169045,13078249961,138919084,6289844496,0.48,0.07,0.07,...,"47,510,326.73","47,232,488.56","-277,838.17",0.55,0.55,"5,718,445,871.14","5,034,762,195.90","4,948,927,442.98","-683,683,675.24","-769,518,428.16"
8,재물보험특약,2018,27,13332444173,5611245445,7721198728,2196004612,0.39,0.24,0.24,...,"2,702,419,554.80","2,547,995,580.24","-154,423,974.56",0.63,0.63,"2,900,530,029.75","1,851,710,996.85","1,851,710,996.85","-1,048,819,032.90","-1,048,819,032.90"
9,재물보험특약,2018,23,7811961357,3768766134,4043195223,1287110924,0.34,0.07,0.08,...,"1,415,118,328.05","1,334,254,423.59","-80,863,904.46",0.41,0.42,"2,098,387,152.02","1,884,682,719.91","1,862,630,087.88","-213,704,432.11","-235,757,064.14"


In [8]:
df3 = df2.groupby(['특약명', '보종코드'])[['수수료증감_보험가격충격', '수수료증감_준비금충격1', '수수료증감_준비금충격2']].sum().reset_index()
df3

,특약명,보종코드,수수료증감_보험가격충격,수수료증감_준비금충격1,수수료증감_준비금충격2
0,근재보험특약,25,"-152,297,490.52","-103,483,634.13","-78,059,248.33"
1,기술보험_ALOP,24,0.00,0.00,0.00
2,기술보험_일반,24,0.00,"-82,047,681.13","-81,854,840.07"
3,기술보험_일반,27,0.00,0.00,0.00
4,배상책임보험특약,26,"-72,345,026.46","-1,441,738,069.18","-1,092,128,730.47"
5,배상책임보험특약,27,"-33,560,463.40","-37,094,653.64","-46,095,549.42"
6,재물보험특약,23,"-81,320,386.66","-794,827,728.59","-901,878,859.00"
7,재물보험특약,27,"-154,701,812.73","-1,747,655,839.58","-1,836,509,366.47"
8,해외PST,24,"-61,733.72",-825.08,-523.29
9,해외PST,27,"-128,876,077.31","-253,668,146.57","-317,956,918.31"


In [9]:
if not any([s == 'result' for s in os.listdir('.')]):
    os.mkdir('result')
    
now = datetime.now().strftime('%Y%m%d%H%M%S')
ver = 'v1.1.3'
with pd.ExcelWriter('./result/일반_충격시나리오_{}_{}.xlsx'.format(ver, now)) as writer:
    df2.to_excel(writer, '일반_충격시나리오_세부내역', index=False)
    df3.to_excel(writer, '일반_충격시나리오', index=False)